# Step 1 - Prepare roads

This notebook merges roads/paths and prepares speeds based on their input data parameters.

It is set up for roads data downloaded from OSM (geofabrik downloads are most typical)

In [ ]:
import os, sys
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
import numpy as np
import pyproj as pyp
from datetime import date
import json

#### Background: essential data prep

Average slope should be added per road segment in QGIS / ArcGIS ahead of time so that the resulting slope categories can be used here. You can use the Add Surface Information tool in ArcGIS to accomplish this (theoretically the SAGA toolkit has similar tools in QGIS). 

#### Load parameters

In [ ]:
data_root = 'D:\\github_test\\'

##################################################################
##################################################################
#read project input parameters that will eventually be passed from the UI
data_file = data_root + 'project_data.json'

##################################################################
##################################################################
#read project variables that will come from UI so that we have our parameters and file locations
with open(data_file, 'rb') as f:
    data_loaded = json.load(f)
f.close()

##################################################################
##################################################################
#read information from the project setup file that's relevant to this section of code
#imports
local_dem_folder = data_loaded['local_dem_folder']
local_roads_folder = data_loaded['local_roads_folder']
local_lc_folder = data_loaded['local_lc_folder']
dest_crs = data_loaded['dest_crs']
dest_crs_id = data_loaded['dest_crs_id']
buffer_m = data_loaded['buffer_m']

##################################################################
##################################################################
seasons = sorted([os.path.join(local_lc_folder,file) \
            for file \
            in os.listdir(local_lc_folder) \
            if file.endswith(".tif")])

for strnum in range(0, len(seasons)):
    seasons[strnum] = str.replace(seasons[strnum], local_lc_folder,"")
    seasons[strnum] = str.replace(seasons[strnum], ".tif","")   

#### OSM roads data

In [ ]:
# Load in the latest OSM data
# Assumes data to have been downloaded from Geofabrik
roads_file = sorted([os.path.join(local_roads_folder,file) \
            for file \
            in os.listdir(local_roads_folder) \
            if file.endswith(".shp")])

roads_file = roads_file[0]
osm = gpd.read_file(roads_file)
osm.crs = dest_crs

# Rename Geofabrik's default 'flcass' column to the standard 'highway'
if 'fclass' in osm.columns:
    osm.rename({'fclass':'highway'},axis=1,inplace=True)

# dicts containing lists of values to replace, with the new key listed lasts
track_dct = dict.fromkeys(['track_grade1','track_grade2','track_grade3','track_grade4','track_grade5'], 'track')
minor_rd_dct = dict.fromkeys(['unclassified','road','service','residential', 'living_street'], 'minor_road')
pth_dct = dict.fromkeys(['path','footway','steps','pedestrian', 'bridleway'], 'path')

# Update the original dict with these new dicts
highway_replace_dct = {}
highway_replace_dct.update(track_dct)
highway_replace_dct.update(minor_rd_dct)
highway_replace_dct.update(pth_dct)

# streamline highway values to a few key types using the above dictionary
osm['highway'] = osm['highway'].replace(highway_replace_dct)

#Filter out any lingering highway types we don't want using a list of values
accepted_road_types = ['path',\
                       'track','minor_road',\
                       'tertiary','secondary','primary','trunk','motorway',\
                       'tertiary_link','secondary_link','primary_link','trunk_link','motorway_link']

osm = osm[osm['highway'].isin(accepted_road_types)]

# dicts containing lists of values to replace, with the new key listed lasts
provincial_dct = dict.fromkeys(['primary','primary_link','trunk','trunk_link','motorway', 'motorway_link'], 'Provincial')
district_dct = dict.fromkeys(['secondary','secondary_link','tertiary','tertiary_link'], 'District')
access_dct = dict.fromkeys(['track'], 'Access')
collector_dct = dict.fromkeys(['minor_road'], 'Collector')

simplified_dct = {}
simplified_dct.update(provincial_dct)
simplified_dct.update(district_dct)
simplified_dct.update(access_dct)
simplified_dct.update(collector_dct)

osm['Road_Class'] = osm['highway'].map(simplified_dct).fillna('Path')
osm['Surface_Final'] = np.nan
osm_slim = osm[['geometry','Road_Class','Surface_Final','Avg_Slope']]

### Manipulate datasets to arrive at final speeds

Slope information has already been joined in to the input shapefiles in a pre-processing step.

Thefore, proceed to use slope to generate terrain category 

In [ ]:
osm_slim['Terrain'] = pd.cut(osm_slim['Avg_Slope'], [-np.inf, 8, 16, np.inf], 
                           labels = ['Plains', 'Hills', 'Mountains']) 

Base speeds will be based on terrain category and road type

In [ ]:
terrain_class_filter = [osm_slim['Terrain'].str.contains('Plains') & osm_slim['Road_Class'].str.contains('Provincial'),    
    osm_slim['Terrain'].str.contains('Plains') & osm_slim['Road_Class'].str.contains('District'),    
    osm_slim['Terrain'].str.contains('Plains') & osm_slim['Road_Class'].str.contains('Access'),    
    osm_slim['Terrain'].str.contains('Plains') & osm_slim['Road_Class'].str.contains('Collector'),
    osm_slim['Terrain'].str.contains('Hills') & osm_slim['Road_Class'].str.contains('Provincial'),    
    osm_slim['Terrain'].str.contains('Hills') & osm_slim['Road_Class'].str.contains('District'),    
    osm_slim['Terrain'].str.contains('Hills') & osm_slim['Road_Class'].str.contains('Access'),    
    osm_slim['Terrain'].str.contains('Hills') & osm_slim['Road_Class'].str.contains('Collector'),
    osm_slim['Terrain'].str.contains('Mountains') & osm_slim['Road_Class'].str.contains('Provincial'),    
    osm_slim['Terrain'].str.contains('Mountains') & osm_slim['Road_Class'].str.contains('District'),    
    osm_slim['Terrain'].str.contains('Mountains') & osm_slim['Road_Class'].str.contains('Access'),    
    osm_slim['Terrain'].str.contains('Mountains') & osm_slim['Road_Class'].str.contains('Collector')]
                        
#read the default speeds assigned to terrain and road type, from the speeds.csv file in the roads folder
csv_file_loc = local_roads_folder + 'speeds.csv'

df_comma = pd.read_csv(csv_file_loc, nrows=1,sep=",")
df_semi = pd.read_csv(csv_file_loc, nrows=1, sep=";")

if df_comma.shape[1]>df_semi.shape[1]:
    speeds_csv =  pd.read_csv(csv_file_loc, sep=',')
else:
    speeds_csv =  pd.read_csv(csv_file_loc, sep=';')
    
speeds_lst = [speeds_csv.loc[0,'Provincial'],speeds_csv.loc[0,'District'],speeds_csv.loc[0,'Access'],speeds_csv.loc[0,'Collector'],\
                 speeds_csv.loc[1,'Provincial'],speeds_csv.loc[1,'District'],speeds_csv.loc[1,'Access'],speeds_csv.loc[1,'Collector'],\
                 speeds_csv.loc[2,'Provincial'],speeds_csv.loc[2,'District'],speeds_csv.loc[2,'Access'],speeds_csv.loc[2,'Collector']]

osm_slim['base_speed'] = np.select(terrain_class_filter,speeds_lst,default=0.0) #default is for path speeds, we make it 0 to avoid vehicles using paths and only proper roads

# must convert the Terrain Type to String to export as a geopackage
osm_slim['Terrain'] = osm_slim['Terrain'].astype(str)

Assign default surface types from the surface.csv file in the roads folder

In [ ]:
csv_file_loc = local_roads_folder + 'surface.csv'

df_comma = pd.read_csv(csv_file_loc, nrows=1,sep=",")
df_semi = pd.read_csv(csv_file_loc, nrows=1, sep=";")

if df_comma.shape[1]>df_semi.shape[1]:
    surface =  pd.read_csv(csv_file_loc, sep=',')
else:
    surface =  pd.read_csv(csv_file_loc, sep=';')

prov_dct = dict.fromkeys(['Provincial'],surface.loc[0,'Surface'])
distr_dct = dict.fromkeys(['District'],surface.loc[1,'Surface'])
access_dct = dict.fromkeys(['Access'],surface.loc[2,'Surface'])
coll_dct = dict.fromkeys(['Collector'],surface.loc[3,'Surface'])
path_dct = dict.fromkeys(['Path'],'Earthen')

osm_slim.Surface_Final = osm_slim.Road_Class
osm_slim.Surface_Final = osm_slim.Surface_Final.replace(prov_dct)
osm_slim.Surface_Final = osm_slim.Surface_Final.replace(distr_dct)
osm_slim.Surface_Final = osm_slim.Surface_Final.replace(access_dct)
osm_slim.Surface_Final = osm_slim.Surface_Final.replace(coll_dct)
osm_slim.Surface_Final = osm_slim.Surface_Final.replace(path_dct)
osm_slim.Surface_Final = osm_slim.Surface_Final.fillna('Earthen')

# assign default road condition based on Road_Class and Terrain

road_condition_filter1 = [osm_slim['Terrain'].str.contains('Plains') & osm_slim['Road_Class'].str.contains('Provincial'),    
    osm_slim['Terrain'].str.contains('Plains') & osm_slim['Road_Class'].str.contains('District'),    
    osm_slim['Terrain'].str.contains('Plains') & osm_slim['Road_Class'].str.contains('Access'),    
    osm_slim['Terrain'].str.contains('Plains') & osm_slim['Road_Class'].str.contains('Collector'),
    osm_slim['Terrain'].str.contains('Hills') & osm_slim['Road_Class'].str.contains('Provincial'),    
    osm_slim['Terrain'].str.contains('Hills') & osm_slim['Road_Class'].str.contains('District'),    
    osm_slim['Terrain'].str.contains('Hills') & osm_slim['Road_Class'].str.contains('Access'),    
    osm_slim['Terrain'].str.contains('Hills') & osm_slim['Road_Class'].str.contains('Collector'),
    osm_slim['Terrain'].str.contains('Mountains') & osm_slim['Road_Class'].str.contains('Provincial'),    
    osm_slim['Terrain'].str.contains('Mountains') & osm_slim['Road_Class'].str.contains('District'),    
    osm_slim['Terrain'].str.contains('Mountains') & osm_slim['Road_Class'].str.contains('Access'),    
    osm_slim['Terrain'].str.contains('Mountains') & osm_slim['Road_Class'].str.contains('Collector')]

csv_file_loc = local_roads_folder + 'default_cond.csv'

df_comma = pd.read_csv(csv_file_loc, nrows=1,sep=",")
df_semi = pd.read_csv(csv_file_loc, nrows=1, sep=";")

if df_comma.shape[1]>df_semi.shape[1]:
    default_cond_csv =  pd.read_csv(csv_file_loc, sep=',')
else:
    default_cond_csv =  pd.read_csv(csv_file_loc, sep=';')

default_cond1 = [default_cond_csv.loc[0,'Provincial'],default_cond_csv.loc[0,'District'],default_cond_csv.loc[0,'Access'],default_cond_csv.loc[0,'Collector'],\
                 default_cond_csv.loc[1,'Provincial'],default_cond_csv.loc[1,'District'],default_cond_csv.loc[1,'Access'],default_cond_csv.loc[1,'Collector'],\
                 default_cond_csv.loc[2,'Provincial'],default_cond_csv.loc[2,'District'],default_cond_csv.loc[2,'Access'],default_cond_csv.loc[2,'Collector']]

# default_cond1
osm_slim['Road_Cond_Final'] = np.select(road_condition_filter1, default_cond1)

### Calculate speeds

In [ ]:
#Specify surface-condition combinations
speed_adj_class_filter = [osm_slim['Surface_Final'].str.contains('Earthen') & osm_slim['Road_Cond_Final'].str.contains('Good'),    
    osm_slim['Surface_Final'].str.contains('Earthen') & osm_slim['Road_Cond_Final'].str.contains('Fair'),    
    osm_slim['Surface_Final'].str.contains('Earthen') & osm_slim['Road_Cond_Final'].str.contains('Poor'),
    osm_slim['Surface_Final'].str.contains('Gravel') & osm_slim['Road_Cond_Final'].str.contains('Good'),    
    osm_slim['Surface_Final'].str.contains('Gravel') & osm_slim['Road_Cond_Final'].str.contains('Fair'),    
    osm_slim['Surface_Final'].str.contains('Gravel') & osm_slim['Road_Cond_Final'].str.contains('Poor'),
    osm_slim['Surface_Final'].str.contains('Paved') & osm_slim['Road_Cond_Final'].str.contains('Good'),    
    osm_slim['Surface_Final'].str.contains('Paved') & osm_slim['Road_Cond_Final'].str.contains('Fair'),    
    osm_slim['Surface_Final'].str.contains('Paved') & osm_slim['Road_Cond_Final'].str.contains('Poor')]

#Season modeling: each season's speeds should also be modified by the surface/condition combination speeds from the [season].csv files in the roads folder
for season_num in range(0, len(seasons)):
    
    current_season = seasons[season_num]
    
    csv_file_loc = local_roads_folder + current_season + '.csv'
        
    df_comma = pd.read_csv(csv_file_loc, nrows=1,sep=",")
    df_semi = pd.read_csv(csv_file_loc, nrows=1, sep=";")

    if df_comma.shape[1]>df_semi.shape[1]:
        season_mods =  pd.read_csv(csv_file_loc, sep=',')
    else:
        season_mods =  pd.read_csv(csv_file_loc, sep=';')

    # list of season speed modifiers
    speed_mods = [season_mods.loc[0,'Good'],season_mods.loc[0,'Fair'],season_mods.loc[0,'Poor'],\
                     season_mods.loc[1,'Good'],season_mods.loc[1,'Fair'],season_mods.loc[1,'Poor'],\
                     season_mods.loc[2,'Good'],season_mods.loc[2,'Fair'],season_mods.loc[2,'Poor']]

    mod_col_name = current_season + '_mod'
    speed_col_name = current_season + '_speed'

    #assign speed_mods to the surface-condition combinations
    osm_slim[mod_col_name] = np.select(speed_adj_class_filter,speed_mods,default=0.0) #default is for path speeds, we make it 0 to avoid vehicles using paths and only proper roads
    osm_slim[speed_col_name] = osm_slim[mod_col_name] * osm_slim.base_speed

## Export final data

Export

In [ ]:
osm_slim.to_file(local_roads_folder + 'final_roads.gpkg',driver='GPKG')